<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/fine_tune_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nn = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
#nn = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
!pip install transformers
!pip install datasets
cats_full = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics',
        'clinical_knowledge', 'college_biology', 'college_chemistry',
        'college_computer_science', 'college_mathematics', 'college_medicine',
        'college_physics', 'computer_security', 'conceptual_physics', 'econometrics',
        'electrical_engineering', 'elementary_mathematics', 'formal_logic',
        'global_facts', 'high_school_biology', 'high_school_chemistry',
        'high_school_computer_science', 'high_school_european_history',
        'high_school_geography', 'high_school_government_and_politics',
        'high_school_macroeconomics', 'high_school_mathematics',
        'high_school_microeconomics', 'high_school_physics',
        'high_school_psychology', 'high_school_statistics',
        'high_school_us_history', 'high_school_world_history',
        'human_aging', 'human_sexuality', 'international_law',
        'jurisprudence', 'logical_fallacies', 'machine_learning',
        'management', 'marketing', 'medical_genetics', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy',
        'prehistory', 'professional_accounting', 'professional_law',
        'professional_medicine', 'professional_psychology', 'public_relations',
        'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

cats = ['abstract_algebra', 'elementary_mathematics', 'formal_logic', 'global_facts',
        'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics',
        'high_school_psychology', 'human_aging', 'human_sexuality',
        'logical_fallacies', 'machine_learning', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'philosophy', 'professional_psychology', 'sociology', 'world_religions']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
"""
from datasets import load_dataset

d = {}
for i in range(len(cats)):
  d[cats[i]] = load_dataset('cais/mmlu', cats[i])
#d.filter(lambda x: x["subject"] in cats)
"""

In [19]:
import pandas as pd
file =('wine-tastings-guide.txt')
df1 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df2 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df3 = pd.read_csv(file, sep = ' - ')
file =('wine-enthusiast.txt') # Questions already framed
df4 = pd.read_csv(file, sep = ' - ')
file =('essence-of-wine.txt') # Questions already framed
df5 = pd.read_csv(file, sep = ' - ')
df1.head()

<ipython-input-19-653f6801f951>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(file, sep = ' - ')
<ipython-input-19-653f6801f951>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv(file, sep = ' - ')
<ipython-input-19-653f6801f951>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df3 = pd.read_csv(file, sep = ' - ')
<ipython-input-19-653f6801f951>:9: ParserWarning: Falling bac

,word,description
0,Austere,"Austere means ""severe or strict in manner, att..."
1,Bouquet,The perfume of a wine. A wine's bouquet is gen...
2,Cassis,One of the most common wine descriptions in ta...
3,Complex,A complex wine is one which has a plethora of ...
4,Concentrated,A concentrated wine is one which is richly fla...


In [20]:
print("total data points available: " + str(df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]))

total data points available: 185


In [21]:
#target format
# how to train on df? yahma/alpaca-cleaned
sample = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")
from random import randint as r
# prompts
imp = ["Tell me more about", "Describe", "I would like to know more about", "I want to know more about",
       "Tell me what", "Explain", "Elaborate", "Enlighten me on", "Educate me on", "Recommend me ", "Give me a recommendation regarding "]
tastes = ["Tell me more about wines that are", "Describe to me wines that feel", "I would like to know more about wines considered", "I want to know more about wines that are",
       "Tell me the meaning of", "Explain", "Elaborate on wines that feel", "Enlighten me on wines that feel", "Educate me on", "I don't understand wines that are", "What does it mean when a wine is"]
what_singular = ["What is", "What's"]
what_plural = ["What are"]
how_singular = ["How does", "How is", "How can", "How come"]
how_plural = ["How do", "How are", "How can", "How come"]
where_singular = ["Where is", "What is the location of", "Where can I find", "How do I come across"]
where_plural = ["Where are", "What are the locations of"]
why_singular = ["Why does", "Why is", "Why would"]
why_plural = ["Why do", "Why are"]
pref = ["I like", "I prefer", "I enjoy", "I love", "I don't mind"]
rank = ["over", "instead of", "less than", "more than", "not as much as", "as much as",
        "almost as much as", "almost as little as", "as little as"]
npref = ["I hate", "I detest", "I can't stand", "I'm wary of", "I don't like"]
point = ["this", "that", "it"]
connect = ["and", "with", "also", "but", "to", "cum"]

# function to construct question/answers from blogs/articles
def construct_full(prompt_set, row, rng):
  if rng == 0:
    instruction = str(prompt_set[r(0, 10)] + " " +row['word']+".")
  elif rng == 1:
    instruction = str(what_singular[r(0, 1)] + " " +row['word']+" in wine terminology?")
  elif rng == 2:
    instruction = str(how_singular[r(0, 3)] + " a wine be " +row['word']+"?")
  else:
    instruction = str(how_singular[r(0, 3)] + " wines feel " +row['word']+"?")
  output = row['description']
  return [instruction, output]

sample.head()

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...


In [22]:
# formatting data into alpaca format
df0 = pd.DataFrame(columns=['instruction', 'input', 'output'])
df1.head()
for index,row in df1.iterrows():
  a = construct_full(tastes, row, r(0,1))
  df0.loc[len(df0)] = [a[0],'',a[1]]
df0

,instruction,input,output
0,What's Austere in wine terminology?,,"Austere means ""severe or strict in manner, att..."
1,Tell me more about wines that are Bouquet.,,The perfume of a wine. A wine's bouquet is gen...
2,What's Cassis in wine terminology?,,One of the most common wine descriptions in ta...
3,What's Complex in wine terminology?,,A complex wine is one which has a plethora of ...
4,What's Concentrated in wine terminology?,,A concentrated wine is one which is richly fla...
5,Elaborate on wines that feel Corpulent.,,Corpulent literally means fat. While this is g...
6,Elaborate on wines that feel Creosote.,,Creosote is a dark brown oil distilled from co...
7,What's Density in wine terminology?,,The density of a wine is how concentrated its ...
8,Explain Depth.,,While depth can be used to refer to the densit...
9,Enlighten me on wines that feel Elegant.,,One of the hallmarks of a great wine is its mo...


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(nn)


def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

#tokenized_datasets = ds.map(tokenize_function, batched=True) ???

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(nn, num_labels=4, torch_dtype="auto")
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
#training_args = TrainingArguments(output_dir="test_trainer")
metric = evaluate.load("accuracy")

In [ ]:
for i in d:
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset= i["test"],
      eval_dataset=i["validation"],
      compute_metrics=compute_metrics)
  trainer.train()